In [18]:
!git clone https://agadetsky@github.com/agadetsky/sochischool.git

Cloning into 'sochischool'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 235 (delta 0), reused 2 (delta 0), pack-reused 231
Receiving objects: 100% (235/235), 50.75 MiB | 32.38 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Checking out files: 100% (194/194), done.


In [10]:
!pip install -qU git+https://github.com/harvardnlp/pytorch-struct

In [1]:
import sys
sys.path.append('/content/sochischool/')

In [2]:
import torch

In [3]:
import listops.data as _data
import listops.data_processing.python.loading as _loading
import listops.model as _model

In [4]:
datasets = _data.get_datasets(
    "var_5_50_nosm_20000",
    datadirpath="/content/sochischool/listops/data_processing/python/listops/data"
  )

/content/sochischool/listops/data_processing/python/listops/data/d2_ml50_nosm
/content/sochischool/listops/data_processing/python/listops/data/d3_ml50_nosm
/content/sochischool/listops/data_processing/python/listops/data/d4_ml50_nosm
/content/sochischool/listops/data_processing/python/listops/data/d5_ml50_nosm
/content/sochischool/listops/data_processing/python/listops/data/d1_ml50_nosm
maxnums
[20000, 2000, 2000]
[2, 3, 4, 5]
file path: /content/sochischool/listops/data_processing/python/listops/data/d2_ml50_nosm/train.tsv
number of skipped sentences due to length > inf: 0
number of skipped sentences due to length < 2: 0
file path: /content/sochischool/listops/data_processing/python/listops/data/d3_ml50_nosm/train.tsv
number of skipped sentences due to length > inf: 0
number of skipped sentences due to length < 2: 0
file path: /content/sochischool/listops/data_processing/python/listops/data/d4_ml50_nosm/train.tsv
number of skipped sentences due to length > inf: 0
number of skipped sen

In [5]:
train_loader, val_loader, test_loader  = _data.get_dataloaders(datasets, batchsize=50)

ds
ds
ds


In [6]:
from listops.data_processing.python.loading import ix_to_word

In [7]:
" ".join([ix_to_word[elem] for elem in x[-6].numpy().tolist() if elem != 15])

NameError: ignored

In [53]:
from torch_struct import DependencyCRF
from listops.model_modules.sampler import DependencySampler, Sampler

In [9]:
class ProjectiveSampler(DependencySampler):

    def __init__(self, noise, tau):
        assert noise in set(['gumbel', 'gaussian'])
        super(ProjectiveSampler, self).__init__("soft", noise, tau, True, False)
        
    def inject_noise(self, A):
        if self.noise == "gumbel":
            u = torch.distributions.utils.clamp_probs(torch.rand_like(A))
            g = u.log().neg().log().neg()
            return (A + g) / self.tau
        elif self.noise == "gaussian":
            pass

    def sample(self, A, lengths, mode):
        if mode == "soft":
            return DependencyCRF(self.inject_noise(A), lengths).marginals
        elif mode == "hard":
            return DependencyCRF(self.inject_noise(A), lengths).argmax.detach()

In [63]:
from listops.model_modules.func_customparse import arcmask_from_lengths
def mask(sample, lengths):
    maxlen = sample.shape[1]
    diag_mask = torch.eye(maxlen, device=sample.device, dtype=bool).unsqueeze(0)
    sample = sample.masked_fill(diag_mask, 0.0)
    arcmask = arcmask_from_lengths(sample, lengths)
    sample = sample.masked_fill(arcmask, 0.0)
    return sample

In [75]:
class IndependentSampler(Sampler):

    def __init__(self, tau):
        super(IndependentSampler, self).__init__("soft", None, tau)

    def forward_train(self, A, lengths=None):
        sample = mask(self.sample(A, lengths, "soft"), lengths)
        return sample

    def forward_eval(self, A, lengths=None):
        sample = mask(self.sample(A, lengths, 'hard'), lengths)
        return sample

    def inject_noise(self, A):
        u = torch.distributions.utils.clamp_probs(torch.rand_like(A))
        logistic = u.log() - u.neg().log1p()
        return (A + logistic) / self.tau

    def sample(self, A, lengths, mode):
        if mode == "soft":
            return self.inject_noise(A).sigmoid()
        elif mode == "hard":
            return (self.inject_noise(A.detach()) > 0.0).float()

In [40]:
sampler = ProjectiveSampler("gumbel", 1.0)

In [41]:
m = _model.get_school_model(sampler)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [42]:
m.cuda()

Model(
  (base): ModelBase(
    (sampler): ProjectiveSampler()
    (computer): KipfMLPGNN(
      (embd): Embedding(16, 60, padding_idx=15)
      (msg_fc1): ModuleList(
        (0): Linear(in_features=120, out_features=60, bias=True)
      )
      (msg_fc2): ModuleList(
        (0): Linear(in_features=60, out_features=60, bias=True)
      )
    )
    (decoder): Decoder(
      (net): Sequential(
        (0): Linear(in_features=60, out_features=60, bias=True)
        (1): ReLU()
        (2): Linear(in_features=60, out_features=60, bias=True)
        (3): ReLU()
        (4): Linear(in_features=60, out_features=10, bias=True)
      )
    )
  )
  (archer): Archer(
    (embd): Embedding(16, 60, padding_idx=15)
    (head_lstm): LSTM(60, 60, batch_first=True, dropout=0.1)
    (head_dropout): Dropout(p=0.1, inplace=False)
    (modif_lstm): LSTM(60, 60, batch_first=True, dropout=0.1)
    (modif_dropout): Dropout(p=0.1, inplace=False)
  )
)

In [43]:
from tqdm import tqdm

In [44]:
opt = torch.optim.AdamW(m.parameters())

In [45]:
m.train()
num_epochs = 2
for _ in range(num_epochs):
    for batch_idx, (x, y, arcs, lengths, depths) in enumerate(tqdm(train_loader)):
        opt.zero_grad()
        bs = x.shape[0]

        x = x.cuda()
        y = y.cuda()
        arcs = arcs.cuda()
        lengths = lengths.cuda()
        
        with torch.set_grad_enabled(True):
            pred_logits = m(x, arcs, lengths)
            loss = torch.nn.functional.cross_entropy(pred_logits, y)
            loss.backward()
            opt.step()
        #print("loss: {:.3f}".format(loss.cpu().item()))

        #acc = (pred_logits.argmax(-1) == y).float().mean()
        #print("acc: {:.3f}".format(acc.cpu().item()))
        #print()

100%|██████████| 2000/2000 [12:27<00:00,  2.68it/s]


In [17]:
def compute_metrics(x, sample, arcs, lengths):
    one = torch.tensor(1.0).cuda() if sample.is_cuda else torch.tensor(1.0)
    zero = torch.tensor(0.0).cuda() if sample.is_cuda else torch.tensor(0.0)
    # Compute true/false positives/negatives for metric calculations.
    maxlen = arcs.shape[-1]
    pad_tn = maxlen - lengths
    tp = torch.where(sample * arcs == 1.0, one, zero).sum((-1, -2))
    tn = torch.where(sample + arcs == 0.0, one, zero).sum((-1, -2)) - pad_tn
    fp = torch.where(sample - arcs == 1.0, one, zero).sum((-1, -2))
    fn = torch.where(sample - arcs == -1.0, one, zero).sum((-1, -2))

    # Calculate IoUs.
    iou = torch.mean((tp) / (tp + fp + fn)).cpu()
    # Calculate precision (attachment).
    precision = torch.mean(tp / (tp + fp)).cpu()
    # Calculate recall.
    recall = torch.mean(tp / (tp + fn)).cpu()
    # Calculate parse accuracy
    parse_acc = (sample == arcs).all(-1).all(-1).float().mean()

    # Clean computations
    # Compute clean attch_score which ignores "]" symbol (requires acces to x)
    close_ix = _loading.word_to_ix[']']
    clean_mask = (x != close_ix).unsqueeze(1).expand_as(arcs) # expands along 2nd dimension
    clean_mask = clean_mask & clean_mask.transpose(1, 2)

    cltp = torch.where((sample * arcs == 1.0) * clean_mask, one, zero).sum((-1, -2))
    cltn = torch.where((sample + arcs == 0.0) * clean_mask, one, zero).sum((-1, -2)) - pad_tn
    clfp = torch.where((sample - arcs == 1.0) * clean_mask, one, zero).sum((-1, -2))
    clfn = torch.where((sample - arcs == -1.0) * clean_mask, one, zero).sum((-1, -2))

    # Calculate IoUs.
    idx = (cltp + clfp + clfn) > 0
    clious = torch.ones_like(cltp)
    clious[idx] = cltp[idx] / (cltp + clfp + clfn)[idx]
    cliou = clious.mean().cpu()
    # Calculate precision (attachment).
    idx = (cltp + clfp) > 0
    clprecisions = torch.zeros_like(cltp)
    clprecisions[idx] = cltp[idx] / (cltp + clfp)[idx]
    clprecision = clprecisions.mean().cpu()
    # Calculate recall.
    idx = (cltp + clfn) > 0
    clrecalls = torch.ones_like(cltp)
    clrecalls[idx] = cltp[idx] / (cltp + clfn)[idx]
    clrecall = clrecalls.mean().cpu()
    # Calculate parse accuracy
    clparse_acc = (sample * clean_mask == arcs * clean_mask).all(-1).all(-1).float().mean()

    if torch.isnan(cliou) or torch.isnan(clprecision) or torch.isnan(clrecall) or torch.isnan(clparse_acc):
        print('Found NaN in cl metrics')

    return iou, cliou, precision, clprecision, recall, clrecall, parse_acc, clparse_acc

In [47]:
m.eval()
val_losses = []
val_accs = []
val_precs = []
val_recs = []
for batch_idx, (x, y, arcs, lengths, depths) in enumerate(tqdm(val_loader)):
    x = x.cuda()
    y = y.cuda()
    arcs = arcs.cuda()
    lengths = lengths.cuda()
    with torch.no_grad():
      pred_logits = m(x, arcs, lengths)
      loss = torch.nn.functional.cross_entropy(pred_logits, y)
      acc = (pred_logits.argmax(-1) == y).float().mean()
      iou, cliou, precision, clprecision, recall, clrecall, parse_acc, clparse_acc = (compute_metrics(x, m.sample, arcs, lengths))

    val_losses.append(loss.item())
    val_accs.append(acc.item())
    val_precs.append(precision.item())
    val_recs.append(recall.item())

100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


In [48]:
import numpy as np

In [49]:
np.mean(val_losses)

0.7222351759672165

In [50]:
np.mean(val_accs)

0.7559000372886657

In [51]:
np.mean(val_precs)

0.520668089389801

In [52]:
np.mean(val_recs)

0.520668089389801

In [86]:
sampler = IndependentSampler(1.0)

In [87]:
m = _model.get_school_model(sampler)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [88]:
m.cuda()

Model(
  (base): ModelBase(
    (sampler): IndependentSampler()
    (computer): KipfMLPGNN(
      (embd): Embedding(16, 60, padding_idx=15)
      (msg_fc1): ModuleList(
        (0): Linear(in_features=120, out_features=60, bias=True)
      )
      (msg_fc2): ModuleList(
        (0): Linear(in_features=60, out_features=60, bias=True)
      )
    )
    (decoder): Decoder(
      (net): Sequential(
        (0): Linear(in_features=60, out_features=60, bias=True)
        (1): ReLU()
        (2): Linear(in_features=60, out_features=60, bias=True)
        (3): ReLU()
        (4): Linear(in_features=60, out_features=10, bias=True)
      )
    )
  )
  (archer): Archer(
    (embd): Embedding(16, 60, padding_idx=15)
    (head_lstm): LSTM(60, 60, batch_first=True, dropout=0.1)
    (head_dropout): Dropout(p=0.1, inplace=False)
    (modif_lstm): LSTM(60, 60, batch_first=True, dropout=0.1)
    (modif_dropout): Dropout(p=0.1, inplace=False)
  )
)

In [89]:
opt = torch.optim.AdamW(m.parameters())

In [90]:
m.train()
num_epochs = 2
for _ in range(num_epochs):
    for batch_idx, (x, y, arcs, lengths, depths) in enumerate(tqdm(train_loader)):
        opt.zero_grad()
        bs = x.shape[0]

        x = x.cuda()
        y = y.cuda()
        arcs = arcs.cuda()
        lengths = lengths.cuda()
        
        with torch.set_grad_enabled(True):
            pred_logits = m(x, arcs, lengths)
            loss = torch.nn.functional.cross_entropy(pred_logits, y)
            loss.backward()
            opt.step()
        #print("loss: {:.3f}".format(loss.cpu().item()))

        #acc = (pred_logits.argmax(-1) == y).float().mean()
        #print("acc: {:.3f}".format(acc.cpu().item()))
        #print()

100%|██████████| 2000/2000 [03:17<00:00, 10.13it/s]


In [91]:
m.eval()
val_losses = []
val_accs = []
val_precs = []
val_recs = []
for batch_idx, (x, y, arcs, lengths, depths) in enumerate(tqdm(val_loader)):
    x = x.cuda()
    y = y.cuda()
    arcs = arcs.cuda()
    lengths = lengths.cuda()
    with torch.no_grad():
      pred_logits = m(x, arcs, lengths)
      loss = torch.nn.functional.cross_entropy(pred_logits, y)
      acc = (pred_logits.argmax(-1) == y).float().mean()
      iou, cliou, precision, clprecision, recall, clrecall, parse_acc, clparse_acc = (compute_metrics(x, m.sample, arcs, lengths))

    val_losses.append(loss.item())
    val_accs.append(acc.item())
    val_precs.append(precision.item())
    val_recs.append(recall.item())

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


In [92]:
np.mean(val_losses)

0.6143985614180565

In [93]:
np.mean(val_accs)

0.7788000345230103

In [94]:
np.mean(val_precs)

0.1949520453810692

In [95]:
np.mean(val_recs)

0.3637669622898102